In [35]:
import sys
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from wavenet import *
from sklearn.model_selection import StratifiedKFold, KFold
import os
from tqdm import tqdm_notebook as tqdm
from torch.utils.data import TensorDataset, DataLoader
%load_ext autoreload
%matplotlib inline
%precision 3
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
print(device)
BATCH_SIZE = 8
SEED = 42

# if you need to reload to apply changeing function, you can use this 
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cpu


In [10]:
B_Z = pd.read_csv("B4_Z.csv", index_col=0).T
feature_cols = list(B_Z.columns[:-1])
target_col = B_Z.columns[-1]
print(feature_cols, target_col)

In [32]:
look_back = 200
x_df = B_Z
x_df = x_df.iloc[:(x_df.shape[0]//look_back)*look_back]
x_df["group"] = x_df.groupby(x_df.index.astype(int) // look_back, sort=False)[0].agg(['ngroup']).values
x_df['group'] = x_df['group'].astype(np.uint16)
X = np.array(list(x_df.groupby('group').apply(lambda x: x[feature_cols].values)))
l = len(X)
train_X = X[:int(l*0.75)]
test_X = X[int(l*0.75):]
y = np.array(list(x_df.groupby('group').apply(lambda x: x[target_col].values)))
train_y = y[:int(l*0.75)]
test_y = y[int(l*0.75):]
print(train_X.shape, test_X.shape)
print(train_y.shape, test_y.shape)

(99, 200, 49) (34, 200, 49)
(99, 200) (34, 200)


In [44]:
class MyDataset(torch.utils.data.Dataset):

    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
train_dataset = MyDataset(train_X, train_y)
test_dataset = MyDataset(test_X, test_y)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [47]:
X, y = train_loader.next()


(99, 200, 49)